In [1]:
import cv2
import torch
import random
import torchreid
import numpy as np
from PIL import Image
from time import time
from glob import glob
from threading import Thread
from datetime import datetime
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models

## P_RI Model

In [3]:
# datamanager = torchreid.data.ImageDataManager(
#     root="reid-data",
#     sources="market1501",
#     targets="market1501",
#     height=256,
#     width=128,
#     batch_size_train=32,
#     batch_size_test=100,
#     transforms=["random_flip", "random_crop"]
# )

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize((256, 128)) #(160, 64)
])

In [4]:
myModel = torchreid.models.build_model(
    name="xception",
    num_classes=751, #datamanager.num_train_pids
    loss="softmax",
    pretrained=True
)

In [5]:
checkpoint = torch.load('model.pth.tar-80')
# checkpoint = torch.load('./log5/HaCNN/model/model.pth.tar-120')
myModel.load_state_dict(checkpoint['state_dict'])

myModel = myModel.cuda()
myModel.eval()

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv2d(128, 128, kernel_size=(3

## Yolo Model 

In [6]:
things_model = torch.hub.load('ultralytics/yolov5', 'custom', path='./yolov5x6.pt',device='cuda:0')
%matplotlib inline

Using cache found in C:\Users\a.nasimi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-2 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


# Camera Settings 

In [7]:
class ThreadedCamera(object):
    def __init__(self, source='cam:Ashkan123@192.168.172.199:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}')

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

# Handeling Functions

In [8]:
def Write_On_Image(label,cor,img_original):
    ROI_Region = [0,0,0,0]
    try: color = rgb_colors[label]
    except KeyError:
        r = random.randrange(1, 256, 20)
        g = random.randrange(1, 256, 20)
        b = random.randrange(1, 256, 20)
        color = (r,g,b)
        rgb_colors[label] = color

    img_original = cv2.rectangle(np.array(img_original), (cor[0]+ROI_Region[0], cor[1]+ROI_Region[1],abs(cor[2]-(cor[0])),abs(cor[3]-(cor[1]))), color, 2)
    img_original = cv2.putText(np.array(img_original), f"No: {label}", (cor[0]+ROI_Region[0], cor[1]+ROI_Region[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2,cv2.LINE_AA)
    
    return img_original

In [9]:
def Pre_Proc_Light(img):
    lightness = np.mean(img)
    if lightness < 95:
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    else: return img

In [10]:
def Person_Detection(img,img_orig,ROI_Region):
    imgs = []
    save_flag = True
    img_original = img_orig.copy()
    
    now_time = datetime.now().strftime("%Y_%m_%d---%I_%M_%S---%p")
    img = np.array(Image.fromarray(img))
    img = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2RGB)
    
    with torch.no_grad(): results = things_model(img)

    if len(results.xyxy[0]) == 0: pass
    else:
        img_original = cv2.cvtColor(np.array(img_original),cv2.COLOR_BGR2RGB)
        
        for indx,res in enumerate(results.xyxy[0]):
            
            predict = results.pandas().xyxy[0]['name'][indx]
            
            if predict == 'person': pass
            else: continue

            res = np.array(res.detach().cpu())
            if res[4]*100 < 45: save_flag = False
            else: 
                x = round(res[0])
                y = round(res[1])
                w = round(res[2])
                h = round(res[3])
                
                imgs.append((img[y:h,x:w],[x,y,h,w]))
        
    return imgs

In [11]:
def Add_Persons(img,thresh=13,flag=False,flag_unknown=False,indx=None):
    img_ = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2RGB)
    img_ = Pre_Proc_Light(img_)
    img_ = transform(img_)
    x = myModel(img_.unsqueeze(0).float().cuda()).detach().cpu()
    
    if flag and not flag_unknown:
        if indx is None: indx = f'{len(database) + 1}'
        else: pass
        
        database.append((indx,x,img))
   
        return indx,None,True
    
    elif flag_unknown and not flag:
        if indx is None: indx = f'{len(database_unknowns) + 1}'
        else: 
            if 'Unknown ' in indx: indx = indx.replace('Unknown ','')
            else: pass
        
        database_unknowns.append((f'Unknown {indx}',x,img))
   
        return f'Unknown {indx}',None,True

    else: return None,None,False

In [12]:
def Person_ReIdentification(img,thresh=13,flag=True,flag_unknown=False):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = Pre_Proc_Light(img)
    img_ = transform(img)
    x = myModel(img_.unsqueeze(0).float().cuda()).detach().cpu()
    
    if flag:
        distns = []
        for indx,embd,_ in database:
            dist = torch.cdist(x, embd, p=2.0)[0][0]
            distns.append((indx,dist))

        try: indx,dist = min(distns, key=lambda x: x[1])
        except ValueError: dist = 9e9

        try:
            if dist < thresh: return indx,dist,True
            else: pass
        except:
            print(dist)
            print(thresh)
            raise "error"
    else: pass
            
    if flag_unknown:
        distns = []
        for indx,embd,_ in database_unknowns:
            dist = torch.cdist(x, embd, p=2.0)[0][0]
            distns.append((indx,dist))

        try: indx,dist = min(distns, key=lambda x: x[1])
        except ValueError: dist = 9e9

        if dist < thresh: return indx,dist,True 
        else: pass
    else: pass
    
    return None,None,False

In [13]:
# from pathlib import Path
# import tensorflow as tf
# from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

# bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))

# def Blurring(image):

# #     image_array = tf.keras.preprocessing.image.img_to_array(image)
#     result = bodypix_model.predict_single(image)

#     # simple mask
#     mask = result.get_mask(threshold=0.20)
#     res = cv2.bitwise_and(image,image,mask = np.uint8(mask))
    
#     blur = cv2.blur(image,(45,45),0)
#     out = image.copy()

#     sharpen_kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
#     sharpen = cv2.filter2D(out, -1, sharpen_kernel)

#     sharpen[res==0] = blur[res==0]
 
#     return sharpen
    

# #     # colored mask (separate colour for each body part)
# #     colored_mask = result.get_colored_part_mask(mask)

# Evaluate On Recorded Video File

In [14]:
ROI_Region,ROI_Person = [0,0,0,0],[0,0,0,0]
img_original,database,database_unknowns,rgb_colors = None,[],[],{}

In [15]:
counter = 0
for indx,pth in enumerate(glob('./Samples_on_Xception/Sample_6/4/Sample1*')):
    if counter > 16: break
    img = cv2.imread(pth)
    imgs = Person_Detection(img,img,ROI_Region)
    
#     img = Blurring(imgs[0][0])
    img = imgs[0][0]
    
    if indx == 0: indx_,_,flag = Add_Persons(img,flag=True)
    else: _,_,flag = Add_Persons(img,flag=True,indx=f'{indx_}')

    if flag: continue
    else: raise 'Error'
    counter += 1

counter = 0       
for indx,pth in enumerate(glob('./Samples_on_Xception/Sample_5/1/Sample2*')):
    if counter > 16: break
    img = cv2.imread(pth)
    imgs = Person_Detection(img,img,ROI_Region)
    
#     img = Blurring(imgs[0][0])
    img = imgs[0][0]
    
    if indx == 0: indx_,_,flag = Add_Persons(img,flag=True)
    else: _,_,flag = Add_Persons(img,flag=True,indx=f'{indx_}')

    if flag: continue
    else: raise 'Error'
    counter += 1
        
counter = 0        
for indx,pth in enumerate(glob('./Samples_on_Xception/Sample_5/1/Sample3*')):
    if counter > 16: break
    img = cv2.imread(pth)
    imgs = Person_Detection(img,img,ROI_Region)
    
#     img = Blurring(imgs[0][0])
    img = imgs[0][0]
    
    if indx == 0: indx_,_,flag = Add_Persons(img,flag=True)
    else: _,_,flag = Add_Persons(img,flag=True,indx=f'{indx_}')

    if flag: continue
    else: raise 'Error'
    counter += 1

In [16]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# cap.release()

# writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_test_on_2_1.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

# for i in range(750,820+1,1):
#     img_original = cv2.imread(f'./Samples_on_Xception/Sample_6/4/frames_test_on_2/{i}.jpg')
#     writer2.write(img_original)
# writer2.release() 

In [17]:
def checkAndGo(i,j,new_frame,old_frame,label,labels,thresh,flag_=False,indx=None,dist=None):
    
    label_ = indx if flag_ else label
    elements = [labels[k] for k, v in enumerate(labels) if v[1] == label_]

    if len(elements) == 0:    # agar label ghablan dide nashode boud  

#         if flag_: nameToLabel[old_frame[i]['label']] = (j,indx,dist)
#         else: pass

        new_frame[j]['label'] = label_
        new_frame[j]['criterion'] = '*'                   
        labels.append((j,label_,label))
        
        return new_frame,old_frame,labels

    else:

        elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == label_] # dar frame e ghbli boude ya na

        if len(elements_) == 0:

            distances_compares = []
            for k,_ in elements:
                _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag=True)
                if flag_: distances_compares.append(dist)
                else:
                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag_unknown=True)
                    if flag_: distances_compares.append(dist)
                    else: 
                        _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh+thresh_param,flag_unknown=True)
                    if flag_: distances_compares.append(dist)
                    else: distances_compares.append(9e9)

            j = np.argmin(distances_compares)

            for k in range(len(elements)):
                if (k == j) and distances_compares[j] != 9e9:
                    new_frame[k]['label'] = label_
#                     try:
                    q = [q for q, v in enumerate(labels) if v[1] == label_][0]
                    labels[q] = (k,label_,label_)
#                     except IndexError: labels.append((j,indx)) 
                    
                    new_frame[k]['criterion'] = '*'
                    
                else:
                    indx,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag_unknown=True)
                    if flag_: new_frame,old_frame,labels = checkAndGo(i,k,new_frame,old_frame,label_,labels,thresh,flag_,indx,dist)
                    else:
                        indx,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh+thresh_param,flag_unknown=True)
                        if flag_: new_frame,old_frame,labels = checkAndGo(i,k,new_frame,old_frame,label_,labels,thresh,flag_,indx,dist)
                        else:
                            indx,_,_ = Add_Persons(new_frame[k]['img'],flag_unknown=True,indx=None)
                            new_frame[k]['label'] = indx
                            new_frame[k]['criterion'] = '*'
                            labels.append((k,indx,label_))
           
            return new_frame,old_frame,labels
        
        else:
            dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
            dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[j]['cor'])))

            if dist_1 <= dist_2 and elements[0][0] != j: 
                indx,_,_ = Add_Persons(new_frame[j]['img'],flag_unknown=True,indx=None)
                new_frame[j]['label'] = indx
                new_frame[j]['criterion'] = '*'
                labels.append((j,indx,'Unknown'))
                
            elif dist_1 <= dist_2 and elements[0][0] == j: 
                new_frame[j]['label'] = old_frame[elements_[0]]['label']
                new_frame[j]['criterion'] = '*'
                for i,_,_ in labels:
                    if i == old_frame[elements_[0]]['label']: flag__ = False
                    else: 
                        flag__ = True
                        break
                        
                if flag__: labels.append((j,old_frame[elements_[0]]['label'],'Unknown'))
                else: pass
                
            else:
                
                new_frame[j]['label'] = old_frame[elements_[0]]['label']
                new_frame[j]['criterion'] = '*'
                
                q = [q for q, v in enumerate(labels) if v[1] == new_frame[elements[0][0]]['label']][0]
                labels[q] = (j,new_frame[j]['label'],new_frame[elements[0][0]]['label'])
                

                indx,dist,flag_ = Person_ReIdentification(new_frame[elements[0][0]]['img'],thresh=thresh,flag_unknown=True)
                if flag_: new_frame,old_frame,labels = checkAndGo(i,elements[0][0],new_frame,old_frame,new_frame[elements[0][0]]['label'],labels,thresh,flag_,indx,dist)
                else:
                    indx,dist,flag_ = Person_ReIdentification(new_frame[elements[0][0]]['img'],thresh=thresh+thresh_param,flag_unknown=True)
                    if flag_: new_frame,old_frame,labels = checkAndGo(i,elements[0][0],new_frame,old_frame,new_frame[elements[0][0]]['label'],labels,thresh,flag_,indx,dist)
                    else:
                        indx,_,_ = Add_Persons(new_frame[elements[0][0]]['img'],flag_unknown=True,indx=None)
                        new_frame[elements[0][0]]['label'] = indx
                        new_frame[elements[0][0]]['criterion'] = '*'
                        labels.append((elements[0][0],indx,new_frame[elements[0][0]]['label']))

            return new_frame,old_frame,labels          

In [18]:
streamer = ThreadedCamera('rtsp:Ashkan321@172.16.60.121:554/Streaming/Channels/001')

In [19]:
thresh = 7.5
thresh_param = 2
thresh_on_estimate = 45

old_frame,last_labels = {},[]
counter,skipped_frame,skipped_count = 0,0,0

while True:
    x,y,h,w = 0,0,0,0
    
    new_frame,labels = {},[]
    
    frame = streamer.grab_frame()
        
    try: img_original = frame.copy()
    except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

    imgs = Person_Detection(frame,img_original,ROI_Region)
    
    if len(imgs) == 0: pass
    else:
        indx_ = 0
        for img,cordinations in imgs:
            x = cordinations[0]
            y = cordinations[1]
            h = cordinations[2]
            w = cordinations[3]
            
#             img_ = Blurring(img)
            img_ = img.copy()
            
            if img_.shape[0] < 180: continue
            elif img_.shape[1] < 110: continue
            else:          
                new_frame[indx_] = {
                    'cor':[x+(w/2),y+(h/2)],
                    'all_cor':[x,y,w,h],
                    'img':img_,
                    'label':'Unknown',
                    'criterion':'-'
                }
                indx_ += 1
        
        if len(new_frame) == 0: 
            counter += 1
            cv2.imshow('Person_ReIdentification', np.array(img_original))

            if cv2.waitKey(1) & 0xFF == ord('q'): 
                cv2.destroyAllWindows()
                streamer.end_frame()
                break            
            
            continue
        else: pass
        
        if len(old_frame) == 0:
            for i in new_frame:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag=True)
                if flag_: Add_Persons(new_frame[i]['img'],flag=True,indx=indx)
                else:      
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag_unknown=True)
                    if flag_: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)  
                    else:
                        indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh+thresh_param,flag_unknown=True)
                        if flag_: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
                        else: indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                            
                new_frame[i]['label'] = indx
                new_frame[i]['criterion'] = '*'
                labels.append((i,indx,'Unknown'))                                
        
        else: pass
        
        nameToLabel = {}
        
        if len(old_frame) <= len(new_frame):      
            
            for i in old_frame:
                distances_compares,flag_ = [],False
                for j in new_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if 'Unknown' in old_frame[i]['label'] or ((counter % thresh_on_estimate) == 0):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh,flag=True) 
                    
                    if flag_: pass
                    else: 
                        if 'Unknown' == old_frame[i]['label']:
                            indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh,flag_unknown=True)

                            if flag_: pass
                            else:
                                indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh+thresh_param,flag_unknown=True)
                                if flag_: pass
                                else: 
                                    indx,_,_ = Add_Persons(new_frame[j]['img'],flag_unknown=True,indx=None)
                                    new_frame[j]['label'] = indx
                                    new_frame[j]['criterion'] = '*'
                                    labels.append((j,indx,old_frame[i]['label']))

                                    continue                                   
                        else: pass 
                    
                else: pass
                
                label = indx if flag_ else old_frame[i]['label']
                new_frame,old_frame,labels = checkAndGo(i,j,new_frame,old_frame,label,labels,thresh,flag_,indx=(indx if flag_ else None),dist=(dist if flag_ else None))

        else:
            for i in new_frame:
                distances_compares,flag_ = [],False
                for j in old_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if 'Unknown' in old_frame[j]['label'] or ((counter % thresh_on_estimate) == 0):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag=True) 
                    
                    if flag_: pass
                    else: 
                        if 'Unknown' == old_frame[j]['label']:
                            indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag_unknown=True)

                            if flag_: pass
                            else:
                                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh+thresh_param,flag_unknown=True)
                                if flag_: pass
                                else: 
                                    indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                                    new_frame[i]['label'] = indx
                                    new_frame[i]['criterion'] = '*'
                                    labels.append((i,indx,old_frame[j]['label']))

                                    continue                                   
                        else: pass 
                    
                else: pass
                
                label = indx if flag_ else old_frame[j]['label']
                new_frame,old_frame,labels = checkAndGo(j,i,new_frame,old_frame,label,labels,thresh,flag_,indx=(indx if flag_ else None),dist=(dist if flag_ else None))     
        
        
        for i in new_frame:
            if new_frame[i]['label'] == 'Unknown':
                indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                new_frame[i]['label'] = indx
                new_frame[i]['criterion'] = '*'
                labels.append((i,indx,'Unknown'))
            elif 'Unknown' in new_frame[i]['label']: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=new_frame[i]['label'])
            else: Add_Persons(new_frame[i]['img'],flag=True,indx=new_frame[i]['label'])
        
            img_original = Write_On_Image(new_frame[i]['label'] ,new_frame[i]['all_cor'],img_original)
        
        
        removed = []
        for i,indx,prv_indx in labels:
            if indx == prv_indx: continue
            else:
                removed_tmp = []
                for label,dist,img in database_unknowns:
                    if label == prv_indx:
                        removed.append((label,dist,img))
                        removed_tmp.append((label,dist,img))
                    else: pass
                    
                for label,_,img in removed_tmp:
                    Add_Persons(img,flag=True,indx=label)

        
        j_1 = [i for i in last_labels if not i in labels]
        j_2 = [i for i in labels if not i in last_labels]
        
        if len(j_1) != 0 and len(j_2) != 0:
            if len(j_1) == len(j_2) and len(j_1) == 1:
                if j_1[0][1] != j_2[0][1]:
                    removed_tmp = []
                    for label,dist,img in database_unknowns:
                        if label == j_1[0][1]:
                            removed.append((label,dist,img))
                            removed_tmp.append((label,dist,img))
                        else: pass

                    for label,_,img in removed_tmp:
                        Add_Persons(img,flag=True,indx=j_2[0][1])

            else: pass
        
            
        for i in removed:
            database_unknowns.remove(i)
        
        last_labels = labels.copy()
        old_frame = new_frame.copy()
        
        print(labels)
        
#         try: database_unknowns = [(indx,embd,img) for indx,embd,img in database_unknowns if not img in removed]
#         except ValueError: database_unknowns = [(indx,embd,img) for indx,embd,img in database_unknowns if not img in removed[0]]
    
    cv2.imshow('Person_ReIdentification', np.array(img_original))
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break

In [20]:
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_1.mp4')
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_1.mp4')


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_12.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

thresh = 7.5
thresh_on_estimate = 6

old_frame,last_labels = {},[]
counter,skipped_frame,skipped_count = 0,0,0

while True:
    x,y,h,w = 0,0,0,0
    
    new_frame,labels = {},[]
    ret,frame = cap.read()
        
    try: img_original = frame.copy()
    except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

    imgs = Person_Detection(frame,img_original,ROI_Region)
    
    if len(imgs) == 0: pass
    else:
        indx_ = 0
        for img,cordinations in imgs:
            x = cordinations[0]
            y = cordinations[1]
            h = cordinations[2]
            w = cordinations[3]
            
#             img_ = Blurring(img)
            img_ = img.copy()
            
            if img_.shape[0] < 180: continue
            elif img_.shape[1] < 110: continue
            else:          
                new_frame[indx_] = {
                    'cor':[x+(w/2),y+(h/2)],
                    'all_cor':[x,y,w,h],
                    'img':img_,
                    'label':'Unknown',
                    'criterion':'-'
                }
                indx_ += 1
        
        if len(new_frame) == 0: 
            counter += 1
            writer2.write(img_original)
            continue
        else: pass
        
        if len(old_frame) == 0:
            for i in new_frame:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag=True)
                if flag_: Add_Persons(new_frame[i]['img'],flag=True,indx=indx)
                else:      
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag_unknown=True)
                    if flag_: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)  
                    else:
                        indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh+4,flag_unknown=True)
                        if flag_: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
                        else: indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                            
                new_frame[i]['label'] = indx
                new_frame[i]['criterion'] = '*'
                labels.append((i,indx,'Unknown'))                                
        
        else: pass
        
        nameToLabel = {}
        
        if len(old_frame) <= len(new_frame):      
            
            for i in old_frame:
                distances_compares,flag_ = [],False
                for j in new_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if 'Unknown' in old_frame[i]['label'] or ((counter % thresh_on_estimate) == 0):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh,flag=True) 
                    
                    if flag_: pass
                    else: 
                        if 'Unknown' == old_frame[i]['label']:
                            indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh,flag_unknown=True)

                            if flag_: pass
                            else:
                                indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh+4,flag_unknown=True)
                                if flag_: pass
                                else: 
                                    indx,_,_ = Add_Persons(new_frame[j]['img'],flag_unknown=True,indx=None)
                                    new_frame[j]['label'] = indx
                                    new_frame[j]['criterion'] = '*'
                                    labels.append((j,indx,old_frame[i]['label']))

                                    continue                                   
                        else: pass 
                    
                else: pass
                
                label = indx if flag_ else old_frame[i]['label']
                new_frame,old_frame,labels = checkAndGo(i,j,new_frame,old_frame,label,labels,thresh,flag_,indx=(indx if flag_ else None),dist=(dist if flag_ else None))

        else:
            for i in new_frame:
                distances_compares,flag_ = [],False
                for j in old_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if 'Unknown' in old_frame[j]['label'] or ((counter % thresh_on_estimate) == 0):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag=True) 
                    
                    if flag_: pass
                    else: 
                        if 'Unknown' == old_frame[j]['label']:
                            indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag_unknown=True)

                            if flag_: pass
                            else:
                                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh+4,flag_unknown=True)
                                if flag_: pass
                                else: 
                                    indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                                    new_frame[i]['label'] = indx
                                    new_frame[i]['criterion'] = '*'
                                    labels.append((i,indx,old_frame[j]['label']))

                                    continue                                   
                        else: pass 
                    
                else: pass
                
                label = indx if flag_ else old_frame[j]['label']
                new_frame,old_frame,labels = checkAndGo(j,i,new_frame,old_frame,label,labels,thresh,flag_,indx=(indx if flag_ else None),dist=(dist if flag_ else None))


#                 elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[j]['label'])]
#                 if len(elements) == 0:    # agar label ghablan dide nashode boud  
                    
# #                     if flag_: nameToLabel[old_frame[j]['label']] = (indx,dist)
# #                     else: pass
                    
#                     label = indx if flag_ else old_frame[j]['label']
#                     new_frame[i]['label'] = label
#                     new_frame[i]['criterion'] = '*'                   
#                     labels.append((i,label))
                    
#                 else:    
#                     elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == (indx if flag_ else None)]

#                     if len(elements_) == 0:
#                         distances_compares = []
#                         for k,_ in elements:
#                             _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag=True)
#                             if flag_: distances_compares.append(dist)
#                             else:
#                                 _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag_unknown=True)
#                                 if flag_: distances_compares.append(dist)
#                                 else: 
#                                     _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh+1,flag_unknown=True)
#                                 if flag_: distances_compares.append(dist)
#                                 else: distances_compares.append(9e9)

#                         j = np.argmin(distances_compares)

#                         for k in range(len(elements)):
#                             if (k == j) and distances_compares[j] != 9e9:
#                                 new_frame[k]['label'] = indx
#                                 q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
#                                 lables[q] = (k,indx)   
#                             else: new_frame[k]['label'] = 'Unknown'

#                             new_frame[k]['status'] = 'P_Id'
#                             new_frame[k]['criterion'] = '*'

#                     else:
#                         dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
#                         dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[i]['cor'])))

#                         if dist_1 <= dist_2: pass
#                         else:
#                             new_frame[i]['label'] = indx
#                             new_frame[i]['criterion'] = '*'

#                             new_frame[elements[0][0]]['label'] = 'Unknown'
#                             new_frame[elements[0][0]]['criterion'] = '*'

#                             q = [q for q, v in enumerate(labels) if v[1] == indx][0]
#                             labels[q] = (i,indx)            
        

        
#         for i in new_frame:
#             if new_frame[i]['label'] == 'Unknown':
#                 indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh,flag_unknown=True)
                
#                 if flag: indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
#                 else:
#                     indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh+1,flag_unknown=True)
#                     if flag: pass
#                     else: indx = None
#                     indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
                
#                 new_frame[i]['label'] = indx
#             else: continue
          
#         nameToLabel,removed = {},[]
#         for i in new_frame:
#             if 'Unknown' in new_frame[i]['label']: 
#                 indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh,flag=True)
#                 if flag:
#                     removed.append(new_frame[i]['img'])
#                     if new_frame[i]['label'] in nameToLabel:
#                         _,dist_ = nameToLabel[new_frame[i]['label']]
#                         if dist_ < dist: pass
#                         else: nameToLabel[new_frame[i]['label']] = (indx,dist)
#                     else: nameToLabel[new_frame[i]['label']] = (indx,dist)
                
#                 else: 
#                     indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh+1,flag=True)
#                     if flag:
#                         removed.append(new_frame[i]['img'])
#                         if new_frame[i]['label'] in nameToLabel:
#                             _,dist_ = nameToLabel[new_frame[i]['label']]
#                             if dist_ < dist: pass
#                             else: nameToLabel[new_frame[i]['label']] = (indx,dist)
#                         else: nameToLabel[new_frame[i]['label']] = (indx,dist)          
#                     else: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=new_frame[i]['label'])
#             else: continue        
        
        
        for i in new_frame:
#             try: 
#                 indx,_ = nameToLabel[new_frame[i]['label']]
#                 new_frame[i]['label'] = indx
#                 Add_Persons(new_frame[i]['img'],indx=indx,flag=True)
#             except KeyError: pass
            if new_frame[i]['label'] == 'Unknown':
                indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                new_frame[i]['label'] = indx
                new_frame[i]['criterion'] = '*'
                labels.append((i,indx,'Unknown'))
            elif 'Unknown' in new_frame[i]['label']: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=new_frame[i]['label'])
            else: Add_Persons(new_frame[i]['img'],flag=True,indx=new_frame[i]['label'])
        
            img_original = Write_On_Image(new_frame[i]['label'] ,new_frame[i]['all_cor'],img_original)
        
        
        removed = []
        for i,indx,prv_indx in labels:
            if indx == prv_indx: continue
            else:
                removed_tmp = []
                for label,dist,img in database_unknowns:
                    if label == prv_indx:
                        removed.append((label,dist,img))
                        removed_tmp.append((label,dist,img))
                    else: pass
                    
                for label,_,img in removed_tmp:
                    Add_Persons(img,flag=True,indx=label)

        
        j_1 = [i for i in last_labels if not i in labels]
        j_2 = [i for i in labels if not i in last_labels]
        
        if len(j_1) != 0 and len(j_2) != 0:
            if len(j_1) == len(j_2) and len(j_1) == 1:
                if j_1[0][1] != j_2[0][1]:
                    removed_tmp = []
                    for label,dist,img in database_unknowns:
                        if label == j_1[0][1]:
                            removed.append((label,dist,img))
                            removed_tmp.append((label,dist,img))
                        else: pass

                    for label,_,img in removed_tmp:
                        Add_Persons(img,flag=True,indx=j_2[0][1])

            else: pass
        
            
        for i in removed:
            database_unknowns.remove(i)
        
        last_labels = labels.copy()
        old_frame = new_frame.copy()
        
        print(labels)
        
#         try: database_unknowns = [(indx,embd,img) for indx,embd,img in database_unknowns if not img in removed]
#         except ValueError: database_unknowns = [(indx,embd,img) for indx,embd,img in database_unknowns if not img in removed[0]]
            
    writer2.write(img_original)
    counter += 1
    
cap.release()
writer2.release()

In [21]:
x = [(0, 'Unknown 1', 'Unknown 1'), (1, 'Unknown 233', 'Unknown 233'), (2, 'Unknown 166', 'Unknown 166'), (3, 'Unknown 321', 'Unknown 321')]
x = [(j,k) for i,j,k in x]
y = [(1, 'Unknown 1', 'Unknown 1'), (2, 'Unknown 233', 'Unknown 233'), (0, '1', '1'), (3, 'Unknown 321', 'Unknown 321')]
y = [(j,k) for i,j,k in y]

In [22]:
j_1 = [i for i in x if not i in y]
j_2 = [i for i in y if not i in x]

In [23]:
j_1

[('Unknown 166', 'Unknown 166')]

In [24]:
j_2

[('1', '1')]

In [25]:
Person_ReIdentification(database_unknowns[464][2],thresh=thresh,flag_unknown=True)

IndexError: list index out of range

In [ ]:
torch.cdist(database_unknowns[464][1], database_unknowns[105][1], p=2.0)[0][0]

In [ ]:
database_unknowns[105][0]

In [ ]:
count = 0
for i,_,_ in database_unknowns:
    if '463' in i:
        print(count)
    else: count += 1

In [ ]:
count

In [ ]:
len(database)

In [ ]:
for i ,_,_ in database_unknowns:
    print(i)

In [ ]:
len(database_unknowns)

In [ ]:
j_1 = [i for i in last_labels if not i in labels]
j_2 = [i for i in labels if not i in last_labels]

In [ ]:
x = [(0, 'Unknown 1', 'Unknown 1')]
y = [(0, 'Unknown 1', 'Unknown 1'), (1, 'Unknown 166', 'Unknown')]


j = [i for i in x if not i in y]
j = [i for i in y if not i in x]

In [ ]:
j

In [ ]:
dist = torch.cdist(x, embd, p=2.0)[0][0]

In [ ]:
labels=[]
old_frame = {}
new_frame = {}

In [ ]:
frame = cv2.imread(f'./Samples_on_Xception/Sample_6/4/frames_test_on_2_labled_10/134.jpg')
imgs = Person_Detection(frame,img_original,ROI_Region)

In [ ]:
indx_ = 0
for img,cordinations in imgs:
    x = cordinations[0]
    y = cordinations[1]
    h = cordinations[2]
    w = cordinations[3]

#             img_ = Blurring(img)
    img_ = img.copy()

    if img_.shape[0] < 190: continue
    elif img_.shape[1] < 110: continue
    else:          
        new_frame[indx_] = {
            'cor':[x+(w/2),y+(h/2)],
            'all_cor':[x,y,w,h],
            'img':img_,
            'label':'Unknown',
            'criterion':'-'
        }
        indx_ += 1

In [ ]:
if len(old_frame) == 0:
    for i in new_frame:
        indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag=True)
        if flag_: Add_Persons(new_frame[i]['img'],flag=True,indx=indx)
        else:      
            indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag_unknown=True)
            if flag_: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)  
            else:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh+1,flag_unknown=True)
                if flag_: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
                else: indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)

        new_frame[i]['label'] = indx
        new_frame[i]['criterion'] = '*'
        labels.append((i,indx))                                

else: pass

In [ ]:
old_frame = new_frame.copy()
new_frame = {}

In [ ]:
labels

In [ ]:
new_frame

In [ ]:
for i in new_frame:
    distances_compares,flag_ = [],False
    for j in old_frame:
        dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
        distances_compares.append(dist_) 

    j = np.argmin(distances_compares)

    if 'Unknown' in old_frame[j]['label'] or ((counter % thresh_on_estimate) == 0):
        indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag=True) 

        if flag_: pass
        else: 
            if 'Unknown' == old_frame[j]['label']:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh,flag_unknown=True)

                if flag_: pass
                else:
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh+1,flag_unknown=True)
                    if flag_: pass
                    else: 
                        indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=None)
                        new_frame[i]['label'] = indx
                        new_frame[i]['criterion'] = '*'
                        labels.append((i,indx))
                        continue                                   
            else: pass 

    else: pass

In [ ]:
labels

In [ ]:
label = indx if flag_ else old_frame[j]['label']
new_frame,old_frame,labels,nameToLabel = checkAndGo(j,i,new_frame,old_frame,label,labels,thresh,nameToLabel,flag_,indx=(indx if flag_ else None),dist=(dist if flag_ else None))

In [ ]:
new_frame

In [ ]:
old_frame

In [ ]:
labels

In [ ]:
Person_ReIdentification(new_frame[0]['img'],thresh=thresh+1,flag_unknown=True)

In [ ]:
distances_compares

In [ ]:
label

In [ ]:
old_frame[0]['label']

In [ ]:
new_frame

In [ ]:
counter = 0
for i,_,_ in database_unknowns:
    if '168' in i: 
        print(i)
        break
    else: counter += 1

In [ ]:
for i,_,_ in database:
    print(i)

In [ ]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_11.mp4')

counter = 0
while True:
    ret,frame = cap.read()
        
    cv2.imwrite(f'./Samples_on_Xception/Sample_6/4/frames_test_on_2_labled_10/{counter}.jpg',frame)
    counter += 1
    
    if ret: continue
    else: break
        
cap.release()

In [ ]:
rgb_colors

In [ ]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')
# # cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_1.mp4')


# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_8.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

# thresh = 7.5
# thresh_on_estimate = 6

# old_frame = {}
# counter,skipped_frame,skipped_count = 0,0,0

# while True:
#     x,y,h,w = 0,0,0,0
    
#     new_frame,labels = {},[]
#     ret,frame = cap.read()
        
#     try: img_original = frame.copy()
#     except AttributeError: break
        
# #     if (skipped_frame % 7) == 0:
# #         if skipped_count < 7:
# #             skipped_count += 1
# # #             writer2.write(img_original)
# #             continue
# #         else:
# #             skipped_count = 0
# #             skipped_frame += 1
# #     else: skipped_frame += 1

#     imgs = Person_Detection(frame,img_original,ROI_Region)
    
#     if len(imgs) == 0: pass
#     else:
#         indx_ = 0
#         for img,cordinations in imgs:
#             x = cordinations[0]
#             y = cordinations[1]
#             h = cordinations[2]
#             w = cordinations[3]
            
# #             img_ = Blurring(img)
#             img_ = img.copy()
            
#             if img_.shape[0] < 120: continue
#             elif img_.shape[1] < 110: continue
#             else:          
#                 new_frame[indx_] = {
#                     'cor':[x+(w/2),y+(h/2)],
#                     'all_cor':[x,y,w,h],
#                     'img':img_,
#                     'label':'Unknown',
#                     'criterion':'-'
#                 }
#                 indx_ += 1
        
#         if len(new_frame) == 0: 
#             counter += 1
#             writer2.write(img_original)
#             continue
#         else: pass
        
#         if len(old_frame) == 0:
#             for i in new_frame:
#                 indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
#                 if flag_: new_frame[i]['label'] = indx
#                 else: pass
#                 new_frame[i]['criterion'] = '*'
#         else: pass
        
        
#         if len(old_frame) <= len(new_frame):      
            
#             for i in old_frame:
#                 distances_compares,flag_ = [],False
#                 for j in new_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
#                     distances_compares.append(dist_) 
  
#                 j = np.argmin(distances_compares)
            
# #                 if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'Unknown'):    

#                 if 'Unknown' in old_frame[i]['label'] or ((counter % thresh_on_estimate) == 0):
#                     indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh,flag=True) 
                    
#                     if flag_: pass
#                     else: 
#                         if 'Unknown' == old_frame[i]['label']:
#                             indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh,flag_unknown=True)

#                             if flag_: pass
#                             else:
#                                 indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh+1,flag_unknown=True)
#                                 if flag_: pass
#                                 else: 
#                                     indx,_,_ = Add_Persons(new_frame[j]['img'],flag_unknown=True,indx=None)
#                                     new_frame[j]['label'] = indx
#                                     new_frame[j]['criterion'] = '*'
#                                     labels.append((j,indx))

#                                     continue                                   
#                         else: pass 
                    
#                 else: pass
                
#                 elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[i]['label'])]

#                 if len(elements) == 0:    # agar label ghablan dide nashode boud  
                    
#                     if flag_: nameToLabel[old_frame[i]['label']] = (indx,dist)
#                     else: pass
                    
#                     label = indx if flag_ else old_frame[i]['label']
#                     new_frame[j]['label'] = label
#                     new_frame[j]['criterion'] = '*'                   
#                     labels.append((j,label))
                    
#                 else:    
#                     elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == (indx if flag_ else None)]

#                     if len(elements_) == 0:
#                         distances_compares = []
#                         for k,_ in elements:
#                             _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag=True)
#                             if flag_: distances_compares.append(dist)
#                             else:
#                                 _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh,flag_unknown=True)
#                                 if flag_: distances_compares.append(dist)
#                                 else: 
#                                     _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh+1,flag_unknown=True)
#                                 if flag_: distances_compares.append(dist)
#                                 else: distances_compares.append(9e9)

#                         j = np.argmin(distances_compares)

#                         for k in range(len(elements)):
#                             if (k == j) and distances_compares[j] != 9e9:
#                                 new_frame[k]['label'] = indx
#                                 q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
#                                 lables[q] = (k,indx)   
#                             else: new_frame[k]['label'] = 'Unknown'

#                             new_frame[k]['status'] = 'P_Id'
#                             new_frame[k]['criterion'] = '*'

#                     else:
#                         dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
#                         dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[j]['cor'])))

#                         if dist_1 <= dist_2: pass
#                         else:
#                             new_frame[j]['label'] = indx
#                             new_frame[j]['criterion'] = '*'

#                             new_frame[elements[0][0]]['label'] = 'Unknown'
#                             new_frame[elements[0][0]]['criterion'] = '*'

#                             q = [q for q, v in enumerate(labels) if v[1] == indx][0]
#                             labels[q] = (j,indx)
                    
        
        
        
#         else:
#             for i in new_frame:
#                 distances_compares,flag_ = [],False
#                 for j in old_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
#                     distances_compares.append(dist_) 
  
#                 j = np.argmin(distances_compares)
            
#                 if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'Unknown'):
# #                 if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[j]['label']):
#                     indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh) 
#                     if flag_: pass
#                     else: continue
#                 else: pass
                
#                 elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[j]['label'])]

#                 if len(elements) == 0:
#                     new_frame[i]['label'] = indx if flag_ else old_frame[j]['label']
#                     new_frame[i]['criterion'] = '*'                    
#                     labels.append((j,old_frame[j]['label']))

#                 else:
#                     if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'Unknown'): pass
# #                     if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[j]['label']): pass
                    
#                     elif old_frame[j]['label'] != 'Unknown':
# #                     elif not ('Unknown' in old_frame[j]['label']):
#                         indx = old_frame[j]['label']
#                         flag_ = True
                    
#                     else: indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)

#                     if flag_:
#                         elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

#                         if len(elements) == 0:
#                             new_frame[i]['label'] = indx
#                             new_frame[i]['criterion'] = '*'
#                             labels.append((i,indx))

#                         else:
#                             elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
#                             if len(elements_) == 0:
#                                 distances_compares = []
#                                 for k,_ in elements:
#                                     _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
#                                     if flag_: distances_compares.append(dist)
#                                     else: distances_compares.append(9e9)

#                                 j = np.argmin(distances_compares)

#                                 for k in range(len(elements)):
#                                     if (k == j) and distances_compares[j] != 9e9:
#                                         new_frame[k]['label'] = indx
#                                         q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
#                                         lables[q] = (k,indx)   
#                                     else: new_frame[k]['label'] = 'Unknown'

#                                     new_frame[k]['status'] = 'P_Id'
#                                     new_frame[k]['criterion'] = '*'
                            
#                             else:
#                                 dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
#                                 dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[i]['cor'])))
                                
#                                 if dist_1 <= dist_2: pass
#                                 else:
#                                     new_frame[i]['label'] = indx
#                                     new_frame[i]['criterion'] = '*'
                                    
#                                     new_frame[elements[0][0]]['label'] = 'Unknown'
#                                     new_frame[elements[0][0]]['criterion'] = '*'
                                    
#                                     q = [q for q, v in enumerate(labels) if v[1] == indx][0]
#                                     labels[q] = (i,indx)
                    
#                     else: pass             
        
#         for i in new_frame:
#             if new_frame[i]['label'] == 'Unknown':
#                 indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh,flag_unknown=True)
                
#                 if flag: indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
#                 else:
#                     indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh+1,flag_unknown=True)
#                     if flag: pass
#                     else: indx = None
#                     indx,_,_ = Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=indx)
                
#                 new_frame[i]['label'] = indx
#             else: continue
          
#         nameToLabel,removed = {},[]
#         for i in new_frame:
#             if 'Unknown' in new_frame[i]['label']: 
#                 indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh,flag=True)
#                 if flag:
#                     removed.append(new_frame[i]['img'])
#                     if new_frame[i]['label'] in nameToLabel:
#                         _,dist_ = nameToLabel[new_frame[i]['label']]
#                         if dist_ < dist: pass
#                         else: nameToLabel[new_frame[i]['label']] = (indx,dist)
#                     else: nameToLabel[new_frame[i]['label']] = (indx,dist)
                
#                 else: 
#                     indx,dist,flag = Person_ReIdentification(new_frame[i]['img'],thresh+1,flag=True)
#                     if flag:
#                         removed.append(new_frame[i]['img'])
#                         if new_frame[i]['label'] in nameToLabel:
#                             _,dist_ = nameToLabel[new_frame[i]['label']]
#                             if dist_ < dist: pass
#                             else: nameToLabel[new_frame[i]['label']] = (indx,dist)
#                         else: nameToLabel[new_frame[i]['label']] = (indx,dist)          
#                     else: Add_Persons(new_frame[i]['img'],flag_unknown=True,indx=new_frame[i]['label'])
#             else: continue        
        
        
#         for i in new_frame:
#             try: 
#                 indx,_ = nameToLabel[new_frame[i]['label']]
#                 new_frame[i]['label'] = indx
#                 Add_Persons(new_frame[i]['img'],indx=indx,flag=True)
#             except KeyError: pass
            
#             img_original = Write_On_Image(new_frame[i]['label'] ,new_frame[i]['all_cor'],img_original)
        
#         old_frame = new_frame.copy()

#         try: database_unknowns = [(indx,embd,img) for indx,embd,img in database_unknowns if not img in removed]
#         except ValueError: database_unknowns = [(indx,embd,img) for indx,embd,img in database_unknowns if not img in removed[0]]
            
#     writer2.write(img_original)
#     counter += 1
    
# cap.release()
# writer2.release()